veri Seti ve ihtiyacimiz olan kutuphanelerin iceriye aktarilmasi

In [1]:
# siniflandirma problemi: iris veri seti 3 farkli sinifa ait siniflandirma problemi
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
data = pd.read_csv('DATA/iris.data',header=None)

In [3]:
# ilk 4 sutunu x degiskenine atar
x = data.iloc[:, :-1].values

In [4]:
# son sutunu y ye kaydeder
y, _ = pd.factorize(data.iloc[:,-1])

In [5]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [6]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [7]:
# train ve test verisi yaratiyoruz
xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size=0.2, random_state=42)

In [8]:
# tensore cevirmek icin bir fonksiyon yazalim
def to_tensor(data, target):
    return torch.tensor(data, dtype=torch.float32), torch.tensor(target, dtype=torch.long)

In [9]:
xTrain, yTrain = to_tensor(xTrain,yTrain)

In [10]:
xTest, yTest= to_tensor(xTest,yTest)

In [11]:
def rbf_kernel(x, centers, beta):
    return torch.exp(-beta * torch.cdist(x, centers)**2)

In [12]:
# RBFN modeli olusturma
class RBFN(nn.Module):
    def __init__(self, num_centers, input_dim, output_dim):
        super(RBFN, self).__init__()
        # RBF merkezlerini rastgele baslat
        self.centers = nn.Parameter(torch.randn(num_centers, input_dim))
        # beta parametresi ile RBF'in genisligini kontrol et
        self.beta = nn.Parameter(torch.ones(1)*2.0)
        # output'u tam baglantili katmana yonlendir
        self.linear = nn.Linear(num_centers, output_dim)

    # ileri yayilim
    def forward(self, x):
        # RBF cekirdek fonksiyonunu hesapla
        phi = rbf_kernel(x, self.centers, self.beta)
        return self.linear(phi)

In [18]:
num_centers = 10

model = RBFN(input_dim = 4, num_centers = num_centers, output_dim =3)

In [20]:
# kayip fonksiyonu tanimlama ve optimizasyon yontemi
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01) 

In [21]:
# modeli egitelim
num_epochs = 100
for epoch in range(num_epochs):
    # gradyanlari sifirla
    optimizer.zero_grad()
    # outputlari bul (prediction yap)
    output = model(xTrain)
    # losss hesapla
    loss = criterion(output,yTrain)
    # geri yayilim
    loss.backward()
    # agirliklari guncelle
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epochs? {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

Epochs? 10/100, Loss: 1.0718
Epochs? 20/100, Loss: 1.0332
Epochs? 30/100, Loss: 0.9729
Epochs? 40/100, Loss: 0.8868
Epochs? 50/100, Loss: 0.7867
Epochs? 60/100, Loss: 0.6825
Epochs? 70/100, Loss: 0.5786
Epochs? 80/100, Loss: 0.4799
Epochs? 90/100, Loss: 0.3953
Epochs? 100/100, Loss: 0.3321


In [22]:
# test and evaluation
with torch.no_grad():
    # test verisi ile tahmin et
    yPred = model(xTest)
    # dogruluk hesapla
    accuracy = (torch.argmax(yPred,axis=1)==yTest).float().mean().item()
    print(f'accuracy: {accuracy}')

accuracy: 1.0
